In [75]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.utils.data

from sklearn.mixture import GaussianMixture 

import matplotlib.pyplot as plt
%matplotlib inline

In [76]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['date'] = pd.to_datetime(data.date)
    print('Days: ',len(set(data.date)))
    return data

In [77]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='amount', index=['date'],
                    columns=['start_id','end_id'], aggfunc=np.sum, fill_value=0)
    return table,table.index.values

In [78]:
def getMatrixData():
    file = dataDir + dataFile
    dataRaw = loadData(file)
    dataTs,dates = getTimeSeries(dataRaw)
    matrix = dataTs.values
    print('Matrix Shape: ',matrix.shape)
    return matrix,dates

In [79]:
dataDir = '/home/urwa/Documents/Projects/AnomalyDetection/Pipeline/data/'
dataFile = '20190402_TaipeiEdgesDatewise.csv'
events_data =dataDir+'TaipeiEvents.csv'

In [80]:
file = dataDir + dataFile
dataRaw = loadData(file)

Raw shape:  (7374816, 5)
Days:  638


In [81]:
dataIn =  pd.pivot_table(dataRaw, values='amount', index=['date'],
                    columns=['end_id'], aggfunc=np.sum, fill_value=0)
dataIn.head(1)

end_id,BL01,BL02,BL03,BL04,BL05,BL06,BL07,BL08,BL09,BL10,...,R20,R21,R22,R22A,R23,R24,R25,R26,R27,R28
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,6326,11444,5696,16834,12256,25559,45609,27781,18745,33240,...,5597,5937,15733,12194,2972,2067,9518,8439,6473,73538


In [82]:
dataOut =  pd.pivot_table(dataRaw, values='amount', index=['date'],
                    columns=['start_id'], aggfunc=np.sum, fill_value=0)
dataOut.head(1)

start_id,BL01,BL02,BL03,BL04,BL05,BL06,BL07,BL08,BL09,BL10,...,R20,R21,R22,R22A,R23,R24,R25,R26,R27,R28
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,6292,10268,5591,15834,12955,26456,42135,29606,18572,34387,...,5734,6801,14919,12373,2943,1957,8887,8603,6870,76751


In [83]:
#matrix = np.stack((dataOut.values, dataIn.values),-1)
matrix = dataOut.values + dataIn.values
matrix = matrix.astype(float)
matrix.shape

(638, 108)

In [84]:
# for i in range(matrix.shape[1]):
#         matrix[:, i] = (matrix[:, i] - np.mean(matrix[:, i])) / (np.std(matrix[:, i]))

In [85]:
for i in range(matrix.shape[0]):
        matrix[i,:] = (matrix[i,:] - np.mean(matrix[i,:])) / (np.std(matrix[i,:]))

In [86]:
stations = list(set(dataRaw.start_id))
n= len(stations)
# edge_index = [[a//n,a%n] for a in range(n*n)]
# edge_index = torch.tensor(edge_index, dtype=torch.long)

In [87]:
dates = list(dataOut.index)
DOW = list(pd.to_datetime(dataOut.index.values).dayofweek)
DOW = ((np.array(DOW) == 5) | (np.array(DOW) == 6)).astype(int)
DOW[:10]

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [88]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(input_dim, hiddenDim[0])
        self.linear2 = nn.Linear(hiddenDim[0], hiddenDim[1])
        self.linear3 = nn.Linear(hiddenDim[1], hiddenDim[2])
        self.linear4 = nn.Linear(hiddenDim[2], n_classes)

        self.init_weights(self.linear1)
        self.init_weights(self.linear2)
        self.init_weights(self.linear3)
        self.init_weights(self.linear4)
        
    def forward(self, x):
        #x, edge_index = data.x, data.edge_index
        x = self.linear1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear4(x)
        return F.log_softmax(x, dim=1)
        
    def init_weights(self,m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    
    def representation(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        return x
            

In [89]:
matrixtensor = torch.tensor(matrix).float()

In [90]:
input_dim = matrix.shape[1]

In [91]:
hiddenDim = [500,100,20]
n_classes = 2

learning_rate=0.001
batch_size = 50
num_epochs = 100

In [92]:
Y = torch.tensor(np.array(DOW), dtype=torch.long)

In [93]:
data_tensor = torch.utils.data.TensorDataset(matrixtensor, Y) 
dataloader = torch.utils.data.DataLoader(dataset = data_tensor, batch_size = batch_size, shuffle = True)

In [94]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model = Net().to(device)
#data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [95]:
for epoch in range(num_epochs):
    for data in dataloader:
        X, y = data
        X = X.cuda()
        y = y.cuda()
#         print(X.shape)
#         print(y.shape)
        # ===================forward=====================
        output = model(X)
        #print(output.shape)
        loss = criterion(output, y)
        #MSE_loss = nn.MSELoss()(output, y)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
        .format(epoch + 1, num_epochs, loss.item()))

epoch [1/100], loss:0.8362
epoch [2/100], loss:0.8508
epoch [3/100], loss:0.6172
epoch [4/100], loss:0.5715
epoch [5/100], loss:0.3872
epoch [6/100], loss:0.5118
epoch [7/100], loss:0.4602
epoch [8/100], loss:0.4868
epoch [9/100], loss:0.4305
epoch [10/100], loss:0.4282
epoch [11/100], loss:0.2819
epoch [12/100], loss:0.3752
epoch [13/100], loss:0.2788
epoch [14/100], loss:0.2689
epoch [15/100], loss:0.2185
epoch [16/100], loss:0.2138
epoch [17/100], loss:0.1943
epoch [18/100], loss:0.3056
epoch [19/100], loss:0.1395
epoch [20/100], loss:0.1420
epoch [21/100], loss:0.2513
epoch [22/100], loss:0.2647
epoch [23/100], loss:0.1323
epoch [24/100], loss:0.1681
epoch [25/100], loss:0.1624
epoch [26/100], loss:0.0821
epoch [27/100], loss:0.0613
epoch [28/100], loss:0.3091
epoch [29/100], loss:0.2862
epoch [30/100], loss:0.2871
epoch [31/100], loss:0.1959
epoch [32/100], loss:0.1000
epoch [33/100], loss:0.2144
epoch [34/100], loss:0.2426
epoch [35/100], loss:0.2181
epoch [36/100], loss:0.2328
e

In [96]:
model.eval()
with torch.no_grad():
    y = np.array(DOW)
    predList = model(matrixtensor.cuda()).cpu().numpy()
    pred = np.argmax(predList,axis=1)    
    print('Accuracy: {:.4f}'.format( np.sum(pred == y) / pred.shape[0] ))

Accuracy: 0.9718


In [97]:
with torch.no_grad():
    representation = model.representation(matrixtensor.cuda()).cpu().numpy()
representation.shape

(638, 20)

In [98]:
# import events data
events_data =dataDir+'TaipeiEvents.csv'
df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)


In [99]:
df_events.head()

,Type,Name,Date
0,National holiday,Republic Day/New Year's Day observed,2017-01-02
1,National holiday,Chinese New Year's Eve,2017-01-27
2,National holiday,Chinese New Year's Day,2017-01-28
3,National holiday,Chinese New Year Holiday 1,2017-01-29
4,National holiday,Chinese New Year Holiday 2,2017-01-30


In [100]:
holidayDates = df_events[df_events.Type == 'National holiday'].Date

In [101]:
holidayDates = [str(d.date()) for d in holidayDates]
dates = [str(d.date()) for d in dates]

In [102]:
anomalyIndex = [i for i,d in enumerate(dates) if d in holidayDates]
len(anomalyIndex)
indexBool = np.array([i in anomalyIndex for i in list(range(matrix.shape[0]))])

In [103]:
def anomalyDetection(y,pval = 0.2,iterN=5,n_com=1):
    #index of regular (non-outlier points)
    #rind=y[:,0]>-10 
    rind = np.array(range(y.shape[0]))
    
    #clustering model
    gm=GaussianMixture(n_components=n_com, n_init=100, max_iter=1000,random_state=0) 
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if(sum(rind)==0):
            print('All anomalies in {} iterations'.format(i+1))
            break
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres

In [104]:
def getResults(reducedMatrix,threshHolds,iterN=5,n_com=1):
    results = []
    for th in threshHolds:
        #th = thres/100
        print("Threshhold: ",th)
        outliers = anomalyDetection(reducedMatrix,th,iterN,n_com)

        tpr = sum(outliers & indexBool)/sum(indexBool)
        fpr = sum(outliers & ~indexBool)/sum(~indexBool)
        precision = sum(outliers & indexBool)/sum(outliers)

        F1 = 2 * (precision * tpr) / (precision + tpr)

        res = {'Cat':'Global', 'th':th, 'TPR':tpr, 'FPR':fpr, 'F1':F1, 'Precision':precision}
        results.append(res)

    resDf = pd.DataFrame(results)    
    return resDf

In [105]:
threshHolds = [0.01, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [106]:
Res1 = getResults(representation,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 

In [107]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.228571,0.003284,0.666667,0.137931,0.01
1,Global,0.500000,0.011494,0.631579,0.413793,0.03
2,Global,0.518519,0.018062,0.560000,0.482759,0.04
3,Global,0.466667,0.027915,0.451613,0.482759,0.05
4,Global,0.597015,0.029557,0.526316,0.689655,0.06
5,Global,0.547945,0.039409,0.454545,0.689655,0.07
6,Global,0.550000,0.047619,0.431373,0.758621,0.08
7,Global,0.478261,0.067323,0.349206,0.758621,0.10
8,Global,0.333333,0.165846,0.204724,0.896552,0.20
9,Global,0.245455,0.269294,0.141361,0.931034,0.30


In [108]:
from sklearn.decomposition import PCA
pca = PCA(n_components=40)
pca.fit(matrix)

PCA(copy=True, iterated_power='auto', n_components=40, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [109]:
reducedMatrixPCA = pca.transform(matrix)
print(reducedMatrixPCA.shape)

(638, 40)


In [110]:
Res1 = getResults(reducedMatrixPCA,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.5
Iteration 1
Iteration

In [111]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.114286,0.006568,0.333333,0.068966,0.01
1,Global,0.375000,0.016420,0.473684,0.310345,0.03
2,Global,0.481481,0.019704,0.520000,0.448276,0.04
3,Global,0.566667,0.022989,0.548387,0.586207,0.05
4,Global,0.537313,0.032841,0.473684,0.620690,0.06
5,Global,0.493151,0.042693,0.409091,0.620690,0.07
6,Global,0.475000,0.052545,0.372549,0.655172,0.08
7,Global,0.456522,0.068966,0.333333,0.724138,0.10
8,Global,0.358974,0.162562,0.220472,0.965517,0.20
9,Global,0.254545,0.267652,0.146597,0.965517,0.30
